<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Método entropía cruzada</center></span>

<figure>
<center>
<img src="../Imagenes/The_Evolution_of_Worlds_0070.jpg" width="800" height="800" align="center"/>
</center>
</figure>


Fuente: [The evolution of the worlds. Wiki commons](https://commons.wikimedia.org/w/index.php?search=evolution+of+worlds&title=Special:MediaSearch&go=Go&type=image)

##   <span style="color:blue">Autores</span>

- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Richard S. Sutton, Andrew G. Barto, Reinforcement learning: an introduction, 2nd edition, 2020](http://library.lol/main/6502B74CE247C4CD4D4FB54747AD7C7E)
1. [Praveen Palanisamy - Hands-On Intelligent Agents with OpenAI Gym_ Your Guide to Developing AI Agents Using Deep Reinforcement Learning, 2020](http://library.lol/main/E4FD128CF9B93E0F7A542B053330517A)
1. [Turing Paper 1936](http://www.thocp.net/biographies/papers/turing_oncomputablenumbers_1936.pdf)
1. [Solving a Reinforcement Learning Problem Using Cross-Entropy Method](https://towardsdatascience.com/solving-a-reinforcement-learning-problem-using-cross-entropy-method-23d9726a737)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [El método de entropía cruzada](#El-método-de-entropía-cruzada)
    * [Conjunto de datos de entrenamiento](#Conjunto-de-datos-de-entrenamiento)
* [El algoritmo de entropía cruzada](#El-algoritmo-de-entropía-cruzada)
* [Los detalles del método de la entropía cruzada](#Los-detalles-del-método-de-la-entropía-cruzada)
* [Modificación del método para el  ambiente FrozenLake](#Modificación-del-método-para-el-ambiente-FrozenLake)

## <span style="color:blue">Introducción</span>

La entropía cruzada se considera un algoritmo evolutivo: algunos individuos se muestrean de una población, y solo los de `élite` gobiernan las características de las generaciones futuras.

Esencialmente, lo que hace el método de `entropía-cruzada`(cross-entropy) es tomar un montón de entradas, ver las salidas producidas, elegir las entradas que han llevado a las mejores salidas y ajustar el agente hasta que estemos satisfechos con las salidas que vemos.

Antes de describir el método técnicamente vamos a repasar los elementos esenciales del aprendizaje reforzado (AR).

<figure>
<center>
<img src="../Imagenes/agente-ambiente.png" width="600" height="400" align="center"/>
</center> 
<caption><center> El proceso del aprendizaje reforzado</center></caption>
</figure>

Fuente: Alvaro Montenegro

Recordamos que tenemos los siguientes elementos esenciales:


- Ambiente: Espacio en donde se encuentra el problema por resolver. El ambiente contiene la información del espacio de observaciones y de las recompensas  que le entrega al agente cada vez que recibe del agente una acción.
- Agente: Objeto que interactúa con el ambiente. El agente recibe del ambiente observaciones y recompensas. Con base en la observación recibida, el agente utiliza la política para decidirse por una acción que le entrega al ambiente. El proceso de intercambio entre el agente y el ambiente puede ocurrir en lotes, es decir, un intercambio de acciones y observaciones entre agente y ambiente puede ocurrir puede constar de varios eventos de intercambios.
- Política: Objeto que recibe como entrada una observación y entrega como salida una distribución de probabilidades para que el agente pueda decidir que acción tomar. En esta lección, la política es implementada como una red neuronal. 
- Acción: Lo que el agente decide en cada momento, de acuerdo con la observación que tiene actualmente del ambiente, la cual procesa con la política.
- Recompensa. Valor numérico que el ambiente le entrega al agente por arribar a un determinado lugar (observación) de acuerdo con la acción que este decidió.

El objetivo del entrenamiento es determinar la política que le permite al agente recibir la mayor recompensa total. 

## <span style="color:blue">El método de entropía cruzada</span>

Recuerde que una política, denotada por $\pi(a|s)$, indica qué acción debe tomar el agente para cada estado observado. En esta lección, consideraremos que el cerebro de nuestro agente será una red neuronal que produce la política.


Nos referimos a los métodos que resuelven este tipo de problemas como `métodos basados en políticas que entrenan la red neuronal que produce la política`. 

En la práctica, la política suele representarse como una distribución de probabilidad sobre las acciones, que el agente puede realizar estando en un estado determinado. Esta característica  convierte nuestro problema en uno muy similar de clasificación, con la cantidad de clases siendo igual a la cantidad de acciones que podemos realizar. En nuestro caso la salida de la red neuronal es un vector que representa una distribución de probabilidad sobre el espacio de las acciones, como se muestra visualmente en la siguiente figura:

<figure>
<center>
<img src="../Imagenes/frozen_action.png" width="800" height="800" align="center"/>
</center> 
</figure>


Fuente: Alvaro Montenegro

En este caso, se dice que tenemos  una política estocástica, porque se devuelve una distribución de probabilidad sobre acciones en lugar de devolver una única acción determinista.

Se busca una política, es decir, una distribución de probabilidad condicional, la cual es inicializada al azar. Con el paso de los pasos de entrenamiento se mejora la política reproduciendo algunos episodios y ajustando la política (parámetro de la red neuronal) de manera que sea más eficiente. Este proceso se repite para que la política mejore gradualmente. Esta es la base del método entropía-cruzada.

### <span style="color:#4CC9F0">Conjunto de datos de entrenamiento</span>

Dado que consideraremos una red neuronal como el cerebro  de este agente, necesitamos encontrar alguna forma de obtener datos que podamos asimilar como un conjunto de datos de entrenamiento, que incluye datos de entrada y sus respectivas etiquetas.


La forma en que vamos a hacer esto es considerando el problema como un problema de aprendizaje supervisado donde los estados observados se consideran las características (datos de entrada) y las acciones constituyen las etiquetas.

Durante la vida del agente, su experiencia se presenta como episodios. Cada episodio es una secuencia de observaciones de los estados que el agente obtuvo del ambiente, las acciones que emitió y las recompensas por estas acciones. 

El núcleo del método entropía-cruzada es descartar los malos episodios y entrenar en los mejores, entonces, 

<center><span style="color:red">  ¿cómo encontramos los mejores? </span><center>
<br>

Imaginemos que nuestro agente ha jugado varios episodios de este tipo. Para cada episodio, podemos calcular el retorno (recompensa total) que ha reclamado el agente. Recuerde que un agente intenta acumular la mayor cantidad posible de recompensas interactuando con el ambiente.



## <span style="color:blue">El algoritmo de entropía cruzada </span>

Desde un punto de vista biológico, el algoritmo de entropía cruzada es un `algoritmo evolutivo`. Algunos individuos son seleccionados de una población y solo los mejores gobiernan las características de las generaciones futuras.

El núcleo del método entropía cruzada es simple. Básicamente consiste en: 

+ generar lotes de episodios, 
+ desechar los *episodios malos* en un lote para entrenar la red neuronal del agente en los mejores episodios. 

Para decidir cuáles desechar, usamos el percentil digamos 70 en nuestro ejemplo, lo que significa que solo conservamos el 30 % que lo hizo mejor que el 70 % de los demás.

Como resultado, la red neuronal aprende a repetir acciones que conducen a que los resultados de las decisiones de la red neuronal sean cada vez mejores a medida que usamos nuevos lotes de episodios de élite. El agente debe ser entrenado hasta alcanzar una determinada recompensa promedio por el umbral del lote de episodios.

Un pseudocódigo del método se puede describir mediante los siguientes pasos:

1. Inicializar el modelo de red neuronal del agente. Cree un lote de episodios que se reproducen en el entorno utilizando nuestro modelo de agente actual.
1. Calcule el retorno esperado para cada episodio y decida un límite de retorno utilizando un percentil de todas las recompensas.
1. Deseche todos los episodios con un retorno por debajo del límite de retorno.
1. Entrene la red neuronal del Agente utilizando los pasos del episodio, es decir, las transiciones *<s,a,r>*) de los episodios "élite" restantes, utilizando el estado *s* como entrada y las acciones emitidas *a* como etiqueta.
1. Repita desde el paso 2 hasta que estemos satisfechos con la recompensa promedio para el lote de episodios.


Una variante del método,  es que podemos mantener los episodios “élite” durante más tiempo. Es importante mencionar que la versión predeterminada del algoritmo muestra episodios del entorno, entrena con los mejores y los descarta. Sin embargo, cuando la cantidad de episodios exitosos es pequeña, los episodios "élite" se pueden mantener por más tiempo, manteniéndolos durante varias iteraciones para entrenar la red neuronal.

## <span style="color:blue">Los detalles del método de la entropía cruzada</span>

Desde un punto de vista biológico, el algoritmo de entropía cruzada es un `algoritmo evolutivo`. Algunos individuos son seleccionado de una población y solo los mejores gobiernan las características de las generaciones futuras.

El núcleo del método entropía cruzada es simple. Básicamente, 

+ se generan lotes de episodios, 
+ se desecha los episodios malos en un lote para entrenar la red neuronal del agente en los mejores episodios. 

Como se puede observar, en la lección [Implementación del Método de entropía cruzada](ar_Metodo_Entropia_Cruzada_implementacion.ipynb), el método funciona bastante bien para el ejemplo *CartPole*. En este caso, se recibe la recompensa en cada paso. Por otro lado, en el ejemplo del ambiente *FrozenLake* el algoritmo termina por número máximo de iteraciones. Vamos a revisar la razón por la cual el método no funciona en este caso. En la siguiente sección se introducen algunas modificaciones que harán que el método converja para el problema FrozenLake. Más adelante retomaremos este ambiente con otro tipo de entrenamiento.  En la siguiente imagen se observan respectivamente la distribución de la recompensa para los casos de *CartPole* y *FrozenLake*. El problema con *FrozenLake* es que la recompensa solamente se entrega al final de cada episodio.


Vamos a describir algunos detalles mínimos del método de entropía cruzada. Para obtener información ampliada del método puede consultar [A Tutorial on the Cross-Entropy Method](http://web.mit.edu/6.454/www/www_fall_2003/gew/CEtutorial.pdf).

A la izquierda en la siguiente imagen se observa el comportamiento de la recompensa para el caso de *CartPole* y a la derecha para *FrozenLake*. Claramente, el comportamiento de *CartPole* es mejor en este caso que el de *FrozenLake*. 

<figure>
<center>
<img src="../Imagenes/distribucion_Reward_CartPole.png" width="400" height="400" align="left"/>
<img src="../Imagenes/distribucion_Reward_FrozenLake.png" width="400" height="400" align="right"/>
</center>
</figure>

Fuente: [Maxim Lapan](http://library.lol/main/2E612EDEF1D325B87C7F5B1FB5542964)

La diferencia en el comportamiento en los dos problema se debe  a que en el caso de *CartPole*, la recompensa es entregada por cada paso en cada episodio; cada episodio entrega valores de recompensa que permite tener una distribución, que puede verse de forma aproximada como normal. No lo es en realidad, pero ciertamente hay valores diferentes para cada episodio, que dan cuenta de como estuvo el agente en el episodio. Por otro lado, *FrozenLake* solamente entrega dos tipos  de recompensa por episodio: uno o cero, según el agente haya arribado a la meta o no. 

El problema es entonces que la selección de episodios *elite* en el caso de *FrozenLake* resulta de muy baja calidad, porque solamente hay dos posibles resultado por episodio.

<figure>
<center>
<img src="../Imagenes/probility_space.png" width="500" height="500" align="centert"/>
</center>
</figure>
<center> Espacio de probabilidad</center>

Fuente: Alvaro Montenegro

El objetivo del algoritmo de AR es estimar la distribución de las acciones en el espacio de las observaciones. Si se dispone de una muestra de la distribución, la estimación es posible mediante el entrenamiento de una red neuronal. Eso es lo que lo se hace exactamente cuando se entrena una red neuronal de clasificación.

El problema que tenemos en aprendizaje reforzado es que tal muestra no está disponible inicialmente. Entonces, el método de entropía cruzada (MEC) provee un procedimiento estocástico para resolver el problema.

El MEC se basa en que en cada paso del algoritmo se obtiene una aproximación mejorada de la distribución de tal manera que cada vez se obtienen mejores muestras, en el sentido que la distribución aproximadamente se parece cada vez más a la distribución de las acciones.

La distribución aproximadamente es calculada por la red neuronal. El propósito de tomar los episodios *élite* por supuesto es mantener aquellos episodios que entregan mayor recompensa. 

El MEC funciona en CarPole, porque los episodios más largos son aquellos en donde el agente usa mejor la información de las observaciones para evitar que la viga caiga. 

El problema en el ambiente FrozeLake, es que la recompensa de cada episodio es 1 o 0, sin importar que sucedió en cada paso del episodio. Como ilustra la imagen de la distribución en este caso. Entonces, no hay realmente forma de obtener episodios élite.

## <span style="color:blue">Modificación del método para el ambiente FrozenLake</span>

Más adelante en este curso volveremos a este ambiente para resolver las limitaciones del MEC con otros métodos de aprendizaje reforzado.

De momento haremos unas mejoras que ayuden al MEC a estimar la distribución de las acciones en el espacio de las observaciones. Haremos los siguiente:

* Lotes de episodios más largos. Pasaremos a 100 episodios por lote.
* Aplicaremos  el factor de descuento $\gamma$ a la recompensa. Así episodios más largos tendrán una menor recompensa y viceversa. Esto incrementa la variabilidad de la distribución de la recompensa.
* Mantendremos episodios élite por más largo tiempo.
* Decreceremos la rata de aprendizaje. Esto implica que la red neuronal tendrá más tiempo para ver en promedio mas muestras de entrenamiento.
* Mucho mayor tiempo de entrenamiento.
